In [3]:
%%capture
%pip install psycopg2-binary pandas

In [4]:
import psycopg2
import pandas as pd

In [13]:
db_config = {
    "host": "54.147.170.202",
    "port": 5432,
    "user": "postgres",
    "password": "postgres",
    "database": "autoprovision",
}

In [14]:
conn = psycopg2.connect(**db_config)
cur = conn.cursor()

In [16]:
list_tables_query = """
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_type = 'BASE TABLE'
ORDER BY table_schema, table_name;
"""
cur.execute(list_tables_query)

tables = cur.fetchall()
print("Tabelas encontradas:")
for schema, table in tables:
	if ['information_schema', 'pg_catalog'].__contains__(schema):
		continue
	print(f"{schema}.{table}")

Tabelas encontradas:
crm.contrato
crm.emprestimo
crm.parcela
parametrizador.modalidade
visualizador.acao
visualizador.grupo_acesso
visualizador.grupo_acesso_permissao_acao
visualizador.permissao
visualizador.usuario


In [10]:
selected_schema = "parametrizador"
selected_table = "modalidade"

query = f"SELECT * FROM {selected_schema}.{selected_table};"
df = pd.read_sql_query(query, conn)
print(f"\nTabela {selected_schema}.{selected_table} carregada em um DataFrame:")
print(df.head())


Tabela parametrizador.modalidade carregada em um DataFrame:
   id                                  nome  taxa_juro  limite_credito  \
0   1       CAPITAL_GIRO_PRAZO_ATE_365_DIAS        6.0          5000.0   
1   2        CREDITO_PESSOAL_NAO_CONSIGNADO        7.5         15000.0   
2   3  CAPITAL_GIRO_PRAZO_SUPERIOR_365_DIAS        5.5         10000.0   
3   4                       CONTA_GARANTIDA        8.0          7000.0   
4   5                       CHEQUE_ESPECIAL        9.0          3000.0   

   maximo_parcelas  
0               12  
1               24  
2               18  
3                6  
4                3  


/tmp/ipykernel_87299/1263776174.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
